# 1. Поднимем pip и установим все зависимости

In [1]:
# 1) Сборка последней версии transformers из исходников:
!pip install --upgrade pip
!pip install git+https://github.com/huggingface/transformers accelerate
# 2) Утилиты для VL:
!pip install qwen-vl-utils[decord]==0.0.8

# Install missing dependency
!pip install gekko

# 3) Наш пакет и зависимости:
%cd /content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation
!pip install -e . \
    --no-deps \
    transformers sentencepiece auto-gptq peft accelerate huggingface-hub \
    diffusers safetensors tqdm pillow qwen-vl-utils



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-iqwl5ix5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-iqwl5ix5
  Resolved https://github.com/huggingface/transformers to commit 0687d481e2c71544501ef9cb3eef795a6e79b1de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 211.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 222.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━

# 2. Подключаем диск и проект в sys.path

In [2]:
import sys
proj = '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation'
if proj not in sys.path:
    sys.path.append(proj)


# 3.Тест Qwen-промптера

https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct

Тестовые челоыечески промпты

- «Сберкот анфас в тёплом золотом свете на закате»
- «Портрет девушки анфас в киновизуальном стиле на фоне городского пейзажа»
- «Кошка-астронавт крупным планом при мягком золотом освещении»
- “A front-view portrait of a vintage car in cinematic color grading during golden hour”
- “Sbercat mascot standing center frame under backlit warm sunlight, photorealistic style”

In [5]:
# 1. Изменили qwen_prompter.py и запушили изменения в GDrive
# 2. В Colab:
%cd /content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation
!pip install -e .  # собрали из вашего обновлённого кода


/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation
Obtaining file:///content/drive/MyDrive/%D0%A1%D0%91%D0%95%D0%A0/RESEACH/Control_Edit_Gen_Image/ranger_generation
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ranger_generation (pyproject.toml) ... done
  Created wheel for ranger_generation: filename=ranger_generation-0.1.0-0.editable-py3-none-any.whl size=4976 sha256=0240dc72b81eccf5fee146b493a805ad8cd46f7cf64f1a9c15432af11cafe0f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-h_edl516/wheels/ca/e7/9a/b209313ef3615048662b640c2f86aade7cb71f2f2d75412894
Successfully built ranger_generation
  Attempting uninstall: ranger_generation
    Found existing installation: ranger_generation 0.1.0
    Uninstalling ranger_generation-0.1.0:
      Successfully uninstalled ranger_generatio

In [1]:
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
from ranger_generation.prompter.parser import detect_rules
detect_rules("A front-view portrait of a vintage car in cinematic color grading during golden hour")


[{'id': 'lighting/golden_hour',
  'images': ['/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation/rules/lighting/golden_hour/images/good/2.jpg',
   '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation/rules/lighting/golden_hour/images/good/4.jpg',
   '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation/rules/lighting/golden_hour/images/good/5.jpg',
   '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation/rules/lighting/golden_hour/images/bad/1.jpg',
   '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation/rules/lighting/golden_hour/images/bad/2.jpg',
   '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation/rules/lighting/golden_hour/images/bad/3.jpg',
   '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ranger_generation/rules/lighting/golden_hour/images/bad/4.jpg',
   '/content/drive/MyDrive/СБЕР/RESEACH/Control_Edit_Gen_Image/ran

In [2]:
def build_flux_prompts(j: dict) -> tuple[str,str]:
    # Попробуем извлечь действие:
    action = j.get("action","").strip()
    scene  = j.get("scene","").strip()
    # 1) Positive:
    if j.get("positive"):
        p = j["positive"].strip()
    else:
        parts = []
        # если есть действие, вставляем его первым
        if action:
            parts.append(action)
        # далее subject
        if j.get("subject"):
            parts.append(j["subject"].strip())
        # потом именно где–как (scene)
        if scene:
            parts.append(scene)
        # стиль, свет, настроение
        for key in ("style","lighting"):
            if j.get(key):
                parts.append(j[key].strip())
        if j.get("mood"):
            parts.append(f"{j['mood'].strip()} mood")
        p = ", ".join(parts)

    # 2) Negative:
    n = j.get("negative","").strip()

    return p, n



In [3]:
from ranger_generation.prompter.qwen_prompter import generate_structured_prompt
import json

import logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s %(message)s")

print()
user_prompt = "Сберкот анфас в тёплом золотом свете на закате"
print(user_prompt)
out_json = generate_structured_prompt(user_prompt)
print(json.dumps(out_json, ensure_ascii=False, indent=2))
print()
pos_prompt, neg_prompt = build_flux_prompts(out_json)
print("Positive:", pos_prompt)
print("Negative:", neg_prompt)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Сберкот анфас в тёплом золотом свете на закате
{
  "subject": "Sbercat",
  "scene": "center frame under warm golden sunset light",
  "style": "photorealistic style",
  "lighting": "warm golden sunset light",
  "mood": "warm and serene",
  "positive": "Sbercat standing center frame under warm golden sunset light, photorealistic style, warm and serene mood, cute grey cartoon cat mascot in a green hoodie, big friendly smile, large expressive green eyes, official Sber corporate palette #00B140, front paws with four fingers, no claws, hind paws with three toes, no visible seams on the single white tooth block, shot from a straight-on perspective, subject centered and facing the camera, symmetrical frontal composition",
  "negative": "lowres, blurry, watermark, unrealistic colors, off-brand bank mascots or competitor logos, wrong hoodie colour, wrong eye colour, extra limbs or fingers, distorted anatomy, text, blur, JPEG artefacts, no top-down (bird's-eye) angle, no low-angle (worm's-eye) s

In [4]:
from ranger_generation.prompter.qwen_prompter import generate_structured_prompt
import json
print()
user_prompt = "Портрет девушки анфас в киновизуальном стиле на фоне городского пейзажа"
print(user_prompt)
out_json = generate_structured_prompt(user_prompt)
print(json.dumps(out_json, ensure_ascii=False, indent=2))
print()
pos_prompt, neg_prompt = build_flux_prompts(out_json)
print("Positive:", pos_prompt)
print("Negative:", neg_prompt)


Портрет девушки анфас в киновизуальном стиле на фоне городского пейзажа
{
  "subject": "girl",
  "scene": "kinovizual style on cityscape background",
  "style": "kinovizual style",
  "lighting": "kinovizual style lighting",
  "mood": "kinovizual style mood",
  "positive": "girl standing in kinovizual style on cityscape background, kinovizual style lighting, kinovizual style mood, shot from a straight-on perspective, subject centered and facing the camera, symmetrical frontal composition",
  "negative": "lowres, blurry, watermark, unrealistic colors, no top-down (bird's-eye) angle, no low-angle (worm's-eye) shot, no side profile, no tilted or Dutch angle, no face obstruction"
}

Positive: girl standing in kinovizual style on cityscape background, kinovizual style lighting, kinovizual style mood, shot from a straight-on perspective, subject centered and facing the camera, symmetrical frontal composition
Negative: lowres, blurry, watermark, unrealistic colors, no top-down (bird's-eye) an

In [5]:
from ranger_generation.prompter.qwen_prompter import generate_structured_prompt
import json
print()
user_prompt = "Кошка-астронавт крупным планом при мягком золотом освещении"
print(user_prompt)
out_json = generate_structured_prompt(user_prompt)
print(json.dumps(out_json, ensure_ascii=False, indent=2))
print()
pos_prompt, neg_prompt = build_flux_prompts(out_json)
print("Positive:", pos_prompt)
print("Negative:", neg_prompt)


Кошка-астронавт крупным планом при мягком золотом освещении
{
  "subject": "cat astronaut",
  "scene": "large close-up shot under soft golden lighting",
  "style": "high-resolution",
  "lighting": "soft golden lighting",
  "mood": "serene and contemplative",
  "positive": "cat astronaut standing large close-up shot under soft golden lighting, high-resolution, serene and contemplative mood",
  "negative": "lowresolution, blurry, dark shadows, unrealistic colors"
}

Positive: cat astronaut standing large close-up shot under soft golden lighting, high-resolution, serene and contemplative mood
Negative: lowresolution, blurry, dark shadows, unrealistic colors


In [6]:
from ranger_generation.prompter.qwen_prompter import generate_structured_prompt
import json
print()
user_prompt = "A front-view portrait of a vintage car in cinematic color grading during golden hour"
print(user_prompt)
out_json = generate_structured_prompt(user_prompt)
print(json.dumps(out_json, ensure_ascii=False, indent=2))
print()
pos_prompt, neg_prompt = build_flux_prompts(out_json)
print("Positive:", pos_prompt)
print("Negative:", neg_prompt)


A front-view portrait of a vintage car in cinematic color grading during golden hour
{
  "subject": "vintage car",
  "scene": "front view, cinematic color grading during golden hour",
  "style": "cinematic color grading",
  "lighting": "golden hour",
  "mood": "warm and nostalgic",
  "positive": "vintage car standing in front view, cinematic color grading during golden hour, warm and nostalgic mood, soft warm golden-hour lighting, long gentle shadows, sun-kissed highlights, low-angle sunlight",
  "negative": "lowres, blurry, watermark, unrealistic colors, no harsh midday sunlight, no cold fluorescent light, no studio strobe flash, no overexposed sky"
}

Positive: vintage car standing in front view, cinematic color grading during golden hour, warm and nostalgic mood, soft warm golden-hour lighting, long gentle shadows, sun-kissed highlights, low-angle sunlight
Negative: lowres, blurry, watermark, unrealistic colors, no harsh midday sunlight, no cold fluorescent light, no studio strobe 

In [7]:
from ranger_generation.prompter.qwen_prompter import generate_structured_prompt
import json
print()
user_prompt = "Sbercat mascot standing center frame under backlit warm sunlight, photorealistic style"
print(user_prompt)
out_json = generate_structured_prompt(user_prompt)
print(json.dumps(out_json, ensure_ascii=False, indent=2))
print()
pos_prompt, neg_prompt = build_flux_prompts(out_json)
print("Positive:", pos_prompt)
print("Negative:", neg_prompt)


Sbercat mascot standing center frame under backlit warm sunlight, photorealistic style
{
  "subject": "Sbercat mascot",
  "scene": "center frame under backlit warm sunlight",
  "style": "photorealistic style",
  "lighting": "backlit warm sunlight",
  "mood": "warm and inviting",
  "positive": "Sbercat mascot standing center frame under backlit warm sunlight, photorealistic style, warm and inviting mood, cute grey cartoon cat mascot in a green hoodie, big friendly smile, large expressive green eyes, official Sber corporate palette #00B140, front paws with four fingers, no claws, hind paws with three toes, no visible seams on the single white tooth block",
  "negative": "lowres, blurry, watermark, unrealistic colors, off-brand bank mascots or competitor logos, wrong hoodie colour, wrong eye colour, extra limbs or fingers, distorted anatomy, text, blur, JPEG artefacts"
}

Positive: Sbercat mascot standing center frame under backlit warm sunlight, photorealistic style, warm and inviting m

In [8]:
from ranger_generation.prompter.qwen_prompter import generate_structured_prompt
import json
print()
user_prompt = "Хочу облако в штанах"
print(user_prompt)
out_json = generate_structured_prompt(user_prompt)
print(json.dumps(out_json, ensure_ascii=False, indent=2))
print()
pos_prompt, neg_prompt = build_flux_prompts(out_json)
print("Positive:", pos_prompt)
print("Negative:", neg_prompt)


Хочу облако в штанах
{
  "subject": "cloud",
  "scene": "wearing cloud in pants",
  "style": "cartoonish style",
  "lighting": "daylight, natural light",
  "mood": "funny and whimsical",
  "positive": "cloud wearing pants, cartoonish style, daylight, natural light, funny and whimsical mood",
  "negative": "lowres, blurry, unrealistic colors"
}

Positive: cloud wearing pants, cartoonish style, daylight, natural light, funny and whimsical mood
Negative: lowres, blurry, unrealistic colors
